In [9]:
#导入相关的包
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import load_dataset

In [10]:
#加载数据集
dataset = load_dataset("csv",data_files="./data.csv",split="train")


In [11]:
#划分数据集
datasets = dataset.train_test_split(test_size=0.1)


In [12]:
#数据集预处理
import torch
tokenizer = AutoTokenizer.from_pretrained("D:/Hugging Face Hub/test2")
def process_function(examples):
    tokenizer_examples = tokenizer(examples["review"], truncation=True, max_length=128)
    tokenizer_examples["label"] = examples["label"]
    return tokenizer_examples
tokenizer_datasets = datasets.map(process_function, batched=True,remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/10788 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

In [13]:
#创建模型
model = AutoModelForSequenceClassification.from_pretrained("D:/Hugging Face Hub/test2")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at D:/Hugging Face Hub/test2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#创建评估函数
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def eval_metric(eval_predict):
    predictions,labels = eval_predict
    predictions = predictions.argmax(dim=-1)
    acc = acc_metric.compute(predictions=predictions, labels=labels)
    f1 = f1_metric.compute(predictions=predictions, labels=labels)
    acc.updata(f1)
    return acc


In [15]:
#创建TrainingArguments
train_args = TrainingArguments(
    output_dir="./checkpoint",
    per_gpu_eval_batch_size=64,
    per_device_train_batch_size=128,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",

)

In [16]:
#创建Trainer
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenizer_datasets["train"],
    eval_dataset=tokenizer_datasets["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric,
)


In [17]:
#模型的训练
trainer.train()

Step,Training Loss


TrainOutput(global_step=255, training_loss=0.30453308703852633, metrics={'train_runtime': 157.1638, 'train_samples_per_second': 205.925, 'train_steps_per_second': 1.623, 'total_flos': 518486028667824.0, 'train_loss': 0.30453308703852633, 'epoch': 3.0})